In [ ]:
import sagemaker as sage

#note this function only works in sage notebooks
role = sage.get_execution_role()

sess = sage.Session()
account = sess.boto_session.client('sts').get_caller_identity()['Account']

# path to the s3 location to store the models
bucket_path = "s3://mdb-sagemaker/models/"
region = sess.boto_session.region_name

# location of the mindsdb container
image = '{}.dkr.ecr.{}.amazonaws.com/mindsdb_lts:latest'.format(account, region)

# note that to_predict is required in hyperparameters
evadb_instance = sage.estimator.Estimator(image,
                    role, 1, 'ml.m4.xlarge',
                    output_path=bucket_path,
                    sagemaker_session=sess,
                    base_job_name="mindsdb-lts-sdk",
                    hyperparameters={"to_predict": "Class"})

# read data from                     
dataset_location = 's3://mdb-sagemaker/diabetes.csv'
evadb_instance.fit(dataset_location)
print('DONE')

# deploy container
predictor = evadb_instance.deploy(1, 'ml.m4.xlarge', endpoint_name='mindsdb-impl')

with open('test_data/diabetes-test.csv', 'r') as reader:
    when_data = reader.read()
result = predictor.predict(when_data).decode('utf-8')
print("Result: ", result)